In [8]:
from pydantic import BaseModel
from typing import List

## Philosopher profile schema
class PhilosopherProfile(BaseModel):
    name: str
    school: str
    stance: str
    core_claims: List[str]
    argumentative_style: str
    primary_goal: str

# Schema for a set of philosophers debating a topic
class PhilosopherSet(BaseModel):
    topic: str
    opposing_topic: str
    philosophers: List[PhilosopherProfile]


In [9]:
from dotenv import load_dotenv
load_dotenv()
model = "openai/gpt-oss-20b"
# initialize the LLM with the model name and any desired parameters

from langchain_groq import ChatGroq

llm = ChatGroq(
    model=model,
    temperature=0.1,
    max_retries=2,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from typing import TypedDict

## Define input state schema
class PhilosophyAgentState(TypedDict, total=False):
    topic: str
    philosopher_set: PhilosopherSet
    enriched_philosophers: list
    history: list
    final_dialogue: str

## philosopher Generator node
prompt = ChatPromptTemplate.from_template("""
You are a philosophy professor.

Given a philosophical concept, do the following:
1. Identify its strongest opposing philosophical position.
2. Create two philosopher profiles:
   - One defending the original concept
   - One defending the opposing concept

Use historical realism when possible.

Concept: {topic}
""")

def create_philosophers(state: PhilosophyAgentState) -> PhilosophyAgentState:
    topic = state.get("topic", "Free Will")
    response = llm.with_structured_output(PhilosopherSet).invoke(
        prompt.format(topic=topic)
    )
    return {**state, "philosopher_set": response}

In [ ]:
from langgraph.graph import StateGraph
## Define the state schema for the graph

## Create the graph with proper state schema
graph_1 = StateGraph(PhilosophyAgentState)
graph_1.add_node("create_philosophers", create_philosophers)
graph_1.set_entry_point("create_philosophers")
graph_1.set_finish_point("create_philosophers")

philosopher_creator = graph_1.compile()


In [17]:
from tavily import TavilyClient
import os

TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
if not TAVILY_API_KEY:
    raise RuntimeError('TAVILY_API_KEY not set in environment')

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

def retrieve_philosophy_knowledge(philosopher):
    query = f"{philosopher.school} philosophy arguments criticisms"
    try:
        docs = tavily_client.search(query, max_results=5)
    except Exception as e:
        docs = []

    return {
        "philosopher": philosopher,
        "sources": docs
    }


In [21]:
def retrieval_map(state):
    philosophers = state["philosopher_set"].philosophers
    enriched = [
        retrieve_philosophy_knowledge(p) for p in philosophers
    ]
    return {"enriched_philosophers": enriched}


In [22]:
class DebateTurn(BaseModel):
    speaker: str
    argument: str
    question: str

def debate_turn(state):
    history = state.get("history", [])
    philosophers = state["enriched_philosophers"]

    prompt = ChatPromptTemplate.from_template("""
You are {name}, a philosopher from the {school} tradition.

Your task:
- Respond to the previous argument
- Defend your philosophical position
- Challenge the opponent
- End with a probing philosophical question

Opponent's last argument:
{last_argument}

Your sources:
{sources}
""")

    turns = []

    for p in philosophers:
        response = llm.with_structured_output(DebateTurn).invoke(
            prompt.format(
                name=p["philosopher"].name,
                school=p["philosopher"].school,
                last_argument=history[-1]["argument"] if history else "Begin the debate",
                sources=p["sources"]
            )
        )
        turns.append(response)

    return {"history": history + turns}

def debate_reduce(state):
    return state  # history already accumulated


In [23]:

## node to format the final dialogue
def format_dialogue(state):
    dialogue = []
    for turn in state["history"]:
        dialogue.append(
            f"{turn.speaker}:\n{turn.argument}\n\nQuestion:\n{turn.question}\n"
        )

    return {
        "final_dialogue": "\n---\n".join(dialogue)
    }


In [ ]:
## Create the final graph
final_graph = StateGraph(PhilosophyAgentState)

final_graph.add_node("philosophers", create_philosophers)
final_graph.add_node("retrieve", retrieval_map)
final_graph.add_node("debate", debate_turn)
final_graph.add_node("format", format_dialogue)

final_graph.set_entry_point("philosophers")
final_graph.add_edge("philosophers", "retrieve")
final_graph.add_edge("retrieve", "debate")
final_graph.add_edge("debate", "format")

philosophy_agent = final_graph.compile()


In [27]:
from IPython.display import display, Markdown

result = philosophy_agent.invoke({
    "topic": "Utilitarianism"
})

# print(result["final_dialogue"])
output = result["final_dialogue"]

display(Markdown(output))

KeyError: 'topic'